---
## Clustering Time Series :  Kmeans


### 🧮 Formule de K-means

La méthode **K-means** vise à regrouper des points en **k clusters** en minimisant la variance intra-cluster. Pour un ensemble de données $\{x_1, x_2, ..., x_n\}$ et des centres de clusters $\{c_1, c_2, ..., c_k\}$, la fonction objectif à minimiser est donnée par :

$$
\text{Argmin}_C \sum_{i=1}^{k} \sum_{x \in C_i} \| x - c_i \|^2
$$

#### Où :
- $C_i$ : le $i$-ème cluster.
- $c_i$ : le centre du $i$-ème cluster (moyenne des points dans le cluster).
- $\| x - c_i \|^2$ : la distance au carré entre un point $x$ et le centre $c_i$ (souvent basée sur la distance Euclidienne).

---

### 🕒 Application aux Séries Temporelles

K-means peut être utilisé pour le **clustering des séries temporelles** en regroupant celles qui partagent des similarités. Cependant, comme K-means est basé sur la distance Euclidienne, il est important d'adapter les données ou de choisir une métrique appropriée.

#### Étapes d'utilisation pour les séries temporelles :
1. **Représentation des séries temporelles :**
   - Utiliser directement les points de la série quand leur longueur est fixe.
   - Extraire des caractéristiques clés (moyenne, variance, autocorrélation, fréquence dominante, etc.).

2. **Choix d'une métrique :**
Par défaut, K-means utilise la distance Euclidienne, mais il existe d'autres métriques comme [**Dynamic Time Warping (DTW)**](https://tslearn.readthedocs.io/en/stable/user_guide/dtw.html) ou des représentations basées sur la fréquence qui peuvent être plus adaptées.


3. **Clustering :**
   - Appliquer K-means pour regrouper les séries temporelles ayant des comportements similaires, tels que des tendances communes, des saisonnalités, ou des amplitudes proches.

>Notre approche vise à **égaler ou surpasser les prédictions** obtenues en traitant la série temporelle globale, tout en prenant en compte les particularités de chaque sous-groupe.


In [1]:
import os 
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler
from scipy.interpolate import interp1d
from sklearn.cluster import KMeans
import plotly.express as px

In [2]:
# --Chemins--
raw_data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'data', 'raw')
processed_data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'data', 'processed')
testing_path = os.path.join(processed_data_path, 'testing.csv')
training_path = os.path.join(processed_data_path, 'training.csv')
stores_path = os.path.join(raw_data_path, 'stores.csv')
# --Tables--
train = pd.read_csv(training_path,parse_dates=['date'])
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d')

test = pd.read_csv(testing_path)
test['date'] = pd.to_datetime(test['date'], format='%Y-%m-%d')

/var/folders/db/swcwxrj54c30_jn72llv15kh0000gn/T/ipykernel_53708/2231735697.py:8: DtypeWarning: Columns (11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(training_path,parse_dates=['date'])
/var/folders/db/swcwxrj54c30_jn72llv15kh0000gn/T/ipykernel_53708/2231735697.py:11: DtypeWarning: Columns (11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(testing_path)


In [3]:
# -- Groupons selon les catégories de produits --

grouped_data = train.groupby(['date', 'family'])['sales'].sum().reset_index()

pivoted_data = grouped_data.pivot(index='family', columns='date', values='sales').fillna(0) # passage des categories en colonnes

In [4]:
data = pivoted_data.select_dtypes(include=[np.number]) #retirer la date

scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

#--Kmeans--

kmeans_model = TimeSeriesKMeans(n_clusters=5, metric='euclidean', n_jobs=-1, max_iter=10, random_state=42)
clusters = kmeans_model.fit_predict(data_scaled)
data['clusters'] = clusters
# Évaluer le clustering
print("Inertia:", kmeans_model.inertia_)  # Inertie intra-cluster

Inertia: 1.5002212365302097


/Users/lilya/Downloads/M2 ISDS/Projects/time_series/venv_time_series/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/lilya/Downloads/M2 ISDS/Projects/time_series/venv_time_series/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [5]:
data= data.reset_index()
data.head()

date,family,2013-01-01 00:00:00,2013-01-02 00:00:00,2013-01-03 00:00:00,2013-01-04 00:00:00,2013-01-05 00:00:00,2013-01-06 00:00:00,2013-01-07 00:00:00,2013-01-08 00:00:00,2013-01-09 00:00:00,...,2015-12-22 00:00:00,2015-12-23 00:00:00,2015-12-24 00:00:00,2015-12-26 00:00:00,2015-12-27 00:00:00,2015-12-28 00:00:00,2015-12-29 00:00:00,2015-12-30 00:00:00,2015-12-31 00:00:00,clusters
0,AUTOMOTIVE,0.0,255.0,161.0,169.0,342.0,360.0,189.0,229.0,164.0,...,962.0,465.0,458.0,609.0,566.0,467.0,470.0,437.0,455.0,1
1,BABY CARE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.0,11.0,14.0,11.0,14.0,13.0,10.0,11.0,5.0,1
2,BEAUTY,2.0,207.0,125.0,133.0,191.0,265.0,124.0,116.0,104.0,...,472.0,258.0,247.0,325.0,365.0,292.0,269.0,289.0,265.0,1
3,BEVERAGES,810.0,72092.0,52105.0,54167.0,77818.0,86184.0,51619.0,46941.0,47910.0,...,379466.0,199461.0,191789.0,214286.0,193941.0,169404.0,182087.0,210801.0,209006.0,2
4,BOOKS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [6]:
# --1er viz__
data['mean_sales'] = data.iloc[:, 3:-1].mean(axis=1)  # Moyenne sur toutes les colonnes avec ventes
fig = px.scatter(
    data,
    x='mean_sales',  # Moyenne des ventes sur l'axe X
    y='clusters',  # Cluster sur l'axe Y
    color='clusters',  # Cluster comme couleur
    text='family',  # Afficher les noms des catégories
    title="Catégories regroupées par clusters avec la moyenne des ventes",
    labels={'mean_sales': 'Moyenne des ventes', 'clusters': 'Clusters'}
)

fig.show()
# --2eme viz__
cluster_counts = data['clusters'].value_counts().reset_index()
cluster_counts.columns = ['Cluster', 'Nombre de Catégories']


fig = px.bar(
    cluster_counts,
    x='Cluster',
    y='Nombre de Catégories',
    text='Nombre de Catégories',  # Ajouter le nombre au-dessus des barres
    title="Nombre de catégories par cluster",
    labels={'Cluster': 'Clusters', 'Nombre de Catégories': 'Nombre de catégories'}
)
fig.show()



>Un clustering par famille de produits ne semble pas être une bonne idée. Essayons plutôt par magasin.


In [7]:
# -- Groupons selon les catégories de magasin  --

grouped_data = train.groupby(['date', 'store_nbr'])['sales'].sum().reset_index()

pivoted_data = grouped_data.pivot(index='store_nbr', columns='date', values='sales').fillna(0)

# -- clustering --

data = pivoted_data.select_dtypes(include=[np.number]) 

scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

#--Kmeans--

kmeans_model = TimeSeriesKMeans(n_clusters=5, metric='euclidean', n_jobs=-1, max_iter=10, random_state=42)
clusters = kmeans_model.fit_predict(data_scaled)
data['clusters'] = clusters
# Évaluer le clustering
print("Inertia:", kmeans_model.inertia_)  # Inertie intra-cluster
data= data.reset_index()

# -- Visualisations --

# --1er viz__
data['mean_sales'] = data.iloc[:, 3:-1].mean(axis=1)  
fig = px.scatter(
    data,
    x='mean_sales',  
    y='clusters',  
    color='clusters',  
    text='store_nbr',  
    title="Catégories regroupées par clusters avec la moyenne des ventes",
    labels={'mean_sales': 'Moyenne des ventes', 'clusters': 'Clusters'}
)

fig.show()
# --2eme viz__
cluster_counts = data['clusters'].value_counts().reset_index()
cluster_counts.columns = ['Cluster', 'Nombre de magasins']


fig = px.bar(
    cluster_counts,
    x='Cluster',
    y='Nombre de magasins',
    text='Nombre de magasins',  # Ajouter le nombre au-dessus des barres
    title="Nombre de magasins par cluster",
    labels={'Cluster': 'Clusters', 'Nombre de magasins': 'Nombre de magasins'}
)
fig.show()

Inertia: 7.335625538164943


/Users/lilya/Downloads/M2 ISDS/Projects/time_series/venv_time_series/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/Users/lilya/Downloads/M2 ISDS/Projects/time_series/venv_time_series/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



>Nous obtenons une meilleure répartition en choisissant un clustering par magasins plutôt que par produits.


In [8]:
# --récuperer les noms des magasins par clusters--
cluster_0 = set(data[data['clusters'] == 0]['store_nbr'])
cluster_1 = set(data[data['clusters'] == 1]['store_nbr'])
cluster_2 = set(data[data['clusters'] == 2]['store_nbr'])
cluster_3 = set(data[data['clusters'] == 3]['store_nbr'])
cluster_4 = set(data[data['clusters'] == 4]['store_nbr'])

clusters = {
    0: cluster_0,
    1: cluster_1,
    2: cluster_2,
    3: cluster_3,
    4: cluster_4,
}


In [10]:
# --training--
clustered_time_series = []

for cluster_id, store_list in clusters.items():
    cluster_data = train[train['store_nbr'].isin(store_list)]  # Filtrer les magasins du cluster
    summed_data = cluster_data.groupby('date')['sales'].sum().reset_index()  
    summed_data['cluster'] = f'Cluster {cluster_id}'  # garder le nom du cluster
    clustered_time_series.append(summed_data)

# Combiner 
all_clusters_time_series_train = pd.concat(clustered_time_series)
# --testing--
clustered_time_series_test = []

for cluster_id, store_list in clusters.items():
    cluster_data = test[test['store_nbr'].isin(store_list)]  # Filtrer les magasins du cluster
    summed_data = cluster_data.groupby('date')['sales'].sum().reset_index()  
    summed_data['cluster'] = f'Cluster {cluster_id}'  # garder le nom du cluster
    clustered_time_series_test.append(summed_data)

# Combiner 
all_clusters_time_series_test = pd.concat(clustered_time_series_test)



In [11]:
fig = px.line(
    all_clusters_time_series_train,
    x='date',
    y='sales',
    color='cluster',
    title='Séries temporelles regroupées par cluster',
    labels={'date': 'Date', 'sales': 'Ventes', 'cluster': 'Cluster'}
)

fig.show()

>On observe des differences entre les series

In [12]:
all_clusters_time_series_train.head() 

,date,sales,cluster
0,2013-01-01,0.000000,Cluster 0
1,2013-01-02,2441.508000,Cluster 0
2,2013-01-03,2589.699001,Cluster 0
3,2013-01-04,2705.500996,Cluster 0
4,2013-01-05,2623.645000,Cluster 0


##### Nous faisons le choix de travailler sur ce DataFrame sans utiliser de variables exogènes, car nous avons constaté qu'elles n'apportaient pas de valeur ajoutée.


In [ ]:
#all_clusters_time_series_train.to_csv('all_clusters_time_series_train.csv', index=False)
#all_clusters_time_series_test.to_csv('all_clusters_time_series_test.csv', index=False)